# Creating and Valuing a CDS Index

Example CDS Index Valuation

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.products.credit import *

####################################################################
# FINANCEPY BETA Version 0.33 - This build:  11 Nov 2023 at 07:47 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



In [3]:
value_date = Date(7, 2, 2006)
settle_date = value_date.add_weekdays(0)
print(settle_date)

07-FEB-2006


## Build Ibor Curve

Build the Ibor curve to value cash flows

In [4]:
depos = []
dcType = DayCountTypes.THIRTY_E_360_ISDA
fixedFreq = FrequencyTypes.SEMI_ANNUAL
swapType = SwapTypes.PAY
swap1 = IborSwap(settle_date,"1Y",swapType,0.0502,fixedFreq,dcType)
swap2 = IborSwap(settle_date,"2Y",swapType,0.0502,fixedFreq,dcType)
swap3 = IborSwap(settle_date,"3Y",swapType,0.0501,fixedFreq,dcType)
swap4 = IborSwap(settle_date,"4Y",swapType,0.0502,fixedFreq,dcType)
swap5 = IborSwap(settle_date,"5Y",swapType,0.0501,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5]

libor_curve = IborSingleCurve(value_date, depos, [], swaps)

## Build Index Curve

We treat an index as a CDS contract with a flat CDS curve at the CDS index spread for the same maturity

In [5]:
step_in_date = value_date

In [6]:
index_coupon = 0.0048375
maturity_date = Date(20, 6, 2010)

cdsMarketContracts = []
cds = CDS(value_date, maturity_date, index_coupon)
cdsMarketContracts.append(cds)

recovery_rate = 0.40
index_curve = CDSCurve(value_date, cdsMarketContracts,libor_curve,recovery_rate)

## Create CDS Index and Perform Valuation

The CDS Index is treated as a CDS with the coupon equal to the CDS Index coupon

In [7]:
cdsRecovery = 0.40
notional = 10.0 * ONE_MILLION
long_protection = True
index_coupon = 0.004 

cdsIndexContract = CDS(step_in_date, maturity_date, index_coupon, notional, long_protection)

In [8]:
spd = cdsIndexContract.par_spread(value_date, index_curve, cdsRecovery) * 10000
print("PAR SPREAD", spd)

PAR SPREAD 48.374972768296715


In [9]:
v = cdsIndexContract.value(value_date, index_curve, cdsRecovery)
print("DIRTY VALUE", v['dirty_pv'])
print("CLEAN VALUE", v['clean_pv'])

DIRTY VALUE 27194.75015949167
CLEAN VALUE 32639.194603936106


In [10]:
p = cdsIndexContract.clean_price(value_date, index_curve, cdsRecovery)
print("CLEAN PRICE", p)

CLEAN PRICE 99.67360805396062


In [11]:
accrued_days = cdsIndexContract.accrued_days()
print("ACCRUED DAYS", accrued_days)

accrued_interest = cdsIndexContract.accrued_interest()
print("ACCRUED COUPON", accrued_interest)

ACCRUED DAYS 49.0
ACCRUED COUPON -5444.444444444444


In [12]:
prot_pv = cdsIndexContract.protection_leg_pv(value_date, index_curve, cdsRecovery)
print("PROTECTION LEG PV", prot_pv)

PROTECTION LEG PV 188528.39213060084


In [13]:
premPV = cdsIndexContract.premium_leg_pv(value_date, index_curve, cdsRecovery)
print("PREMIUM LEG PV", premPV)

PREMIUM LEG PV 161333.64197110917


In [14]:
rpv01 = cdsIndexContract.risky_pv01(value_date, index_curve)
print("DIRTY RPV01", rpv01['dirty_rpv01'])
print("CLEAN RPV01", rpv01['clean_rpv01'])

DIRTY RPV01 4.0333410492777295
CLEAN RPV01 3.8972299381666184


Copyright (c) 2020 Dominic O'Kane